# Person Detector #

This script is used to find the person in the image so that the person can be cropped out and be used in the project. This will create a more accurate result.

### Import dependencies ###

In [1]:
import cv2
import numpy as np
import tensorflow as tf
from yolov3.utils import Load_Yolo_model, image_preprocess, postprocess_boxes, nms
from yolov3.configs import *

### Import images ###

In [84]:
image = "./reference_pose/c_migi_gedan_barai.png"

### Crop image function ###

In [82]:
def crop(image, bboxes):
    for bbox in bboxes:
        coor = np.array(bbox[:4], dtype=np.int32)
        (x1, y1), (x2, y2) = (coor[0], coor[1]), (coor[2], coor[3])

        # crop image
        cropped_image = image[y1-30:y2+30, x1-30:x2+30]

    return cropped_image

In [80]:
def detect_and_crop_image(Yolo, image_path, output_path, input_size=416, show=False, score_threshold=0.3, iou_threshold=0.45):
    original_image = cv2.imread(image_path)
    original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
    original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

    image_data = image_preprocess(np.copy(original_image), [
                                  input_size, input_size])
    image_data = image_data[np.newaxis, ...].astype(np.float32)

    if YOLO_FRAMEWORK == "tf":
        pred_bbox = Yolo.predict(image_data)
    elif YOLO_FRAMEWORK == "trt":
        batched_input = tf.constant(image_data)
        result = Yolo(batched_input)
        pred_bbox = []
        for key, value in result.items():
            value = value.numpy()
            pred_bbox.append(value)

    pred_bbox = [tf.reshape(x, (-1, tf.shape(x)[-1])) for x in pred_bbox]
    pred_bbox = tf.concat(pred_bbox, axis=0)

    bboxes = postprocess_boxes(
        pred_bbox, original_image, input_size, score_threshold)
    bboxes = nms(bboxes, iou_threshold, method='nms')

    image = crop(original_image, bboxes)
    # CreateXMLfile("XML_Detections", str(int(time.time())), original_image, bboxes, read_class_names(CLASSES))

    if output_path != '':
        cv2.imwrite(output_path, image)
    if show:
        # Show the image
        cv2.imshow("Predicted image", image)
        # Load and hold the image
        cv2.waitKey(0)
        # To close the window after the required kill value was provided
        cv2.destroyAllWindows()

    return image

### Load yolo model ###

In [4]:
yolo = Load_Yolo_model()

### Detect and crop images ###

In [85]:
detect_and_crop_image(yolo, image, '', show=True)

1/1 [==============================] - 0s 493ms/step


array([[[ 56,  75,  91],
        [ 56,  75,  91],
        [ 56,  75,  91],
        ...,
        [ 59,  77,  98],
        [ 59,  77,  98],
        [ 59,  77,  98]],

       [[ 56,  75,  91],
        [ 56,  75,  91],
        [ 56,  75,  91],
        ...,
        [ 59,  77,  98],
        [ 59,  77,  98],
        [ 59,  77,  98]],

       [[ 56,  75,  91],
        [ 56,  75,  91],
        [ 56,  75,  91],
        ...,
        [ 58,  76,  97],
        [ 58,  76,  97],
        [ 58,  76,  97]],

       ...,

       [[137, 188, 226],
        [137, 188, 226],
        [137, 188, 226],
        ...,
        [131, 186, 222],
        [132, 187, 223],
        [132, 187, 223]],

       [[137, 187, 226],
        [137, 187, 226],
        [136, 186, 225],
        ...,
        [131, 186, 222],
        [132, 187, 223],
        [132, 187, 223]],

       [[136, 186, 225],
        [136, 186, 225],
        [136, 186, 225],
        ...,
        [132, 187, 223],
        [132, 187, 223],
        [133, 188, 224]]